In [174]:
#! pip install matplotlib spacy nltk gensim numpy scipy scikit-learn 

In [175]:
#!python -m spacy download de_core_news_lg

In [176]:
#!curl -O https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt

In [177]:
#!curl -O https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vocab.txt

In [178]:
!pip install pandas chardet

python3(64044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [179]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import re
import os
import chardet


from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/galina.ryazanskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/galina.ryazanskaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [180]:
#  Function to flatten elements of different lists in one list of lists 'l' into a single list
flatten = lambda l: [item for sublist in l for item in sublist]

In [181]:
# change your directories here
IN_DIR = '/Users/galina.ryazanskaya/Downloads/thesis?/code?/transcripts_NET/in/' 
OUT_DIR_TRANSCRIPTS = '/Users/galina.ryazanskaya/Downloads/thesis?/code?/transcripts_NET/out/'
OUT_DIR = './'

<h1>Preprocessing</h1>

<h3>Preparing preprocessing</h3>

In [182]:
# Function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        detector = chardet.universaldetector.UniversalDetector()
        for line in f:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
        return detector.result['encoding']

In [183]:
#Define list of stopwords and fillers
stopwords = stopwords.words('german')
fillers = ['Ja', 'ja', 'Joa', 'joa', 'Ähm', 'ähm', 'Äh', 'äh', 'Oh', 'oh', 'Ohje', 'ohje',  'Hm', 'hm', 'Mh', 'mh', 'Mhm', 'mhm', 'Na', 'na', 'Naja', 'naja', 'Ne', 'ne', 'EM', 'em', 'Ehm', 'ehm', '[unv]', '(...)', '(..)', '(.)', '[...]', '[..]', '[.]']

In [184]:
#Define dictionary of encoding errors to use in replace_chars function

encoding_errors_dict = {'Ã¼':'ü', 'Ãœ':'Ü', 'Ã¤':'ä', 'Ã„':'Ä', 'Ã¶':'ö', 'Ã–':'Ö', 'ÃŸ':'ß', 'Ÿ':'ü', '§':'ß', '€':'ä', 'Š':'ä', 'š':'ö', 'д': 'ä', 'ь':'ü', 'ц': 'ö', 'Я': 'ß', 'Д': 'Ä', 'Ь': 'Ü', 'Ц': 'Ö', 'й': 'é', 'Й': 'É', 'б': 'à', 'Б': 'À'}

<h3>All preprocessing functions</h3>

In [185]:
punct = '\!"#$%&\'\(\)\*\+,\-\./:;<=>\?@[\\]^_`{|}~\^„“/…'

def test_unicode(text):
    non_unicode = re.findall(f'[^A-Za-zÀ-ž0-9\\n {punct}]', text)
    return not bool(non_unicode), non_unicode

In [288]:
#  Function designed to replace specific characters in a given text with corresponding replacement characters defined in a dictionary (or mapping)
def replace_chars(text, char_map):
    for char in char_map:
        text = text.replace(char, char_map[char])
    return text

def multiple_replace(text, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in rep_dict]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)], text)

#Remove timestamps
def remove_timestamps(text):
    return re.sub('#\d\d:\d\d:\d\d-\d#', '', text)

# function to take a text, tokenize it, preprocess, remove stopwords
def clean_stopwords(text, stopwords=[]):
    tokens = nltk.word_tokenize(text) # tokenize text
    tokens = [w for w in tokens if not w in stopwords] # remove stopwords
    text_cleaned = ' '.join(tokens)
    return text_cleaned

# function to take a text, tokenize it, preprocess, remove fillers
def clean_fillers(text, fillers=[]):
    tokens = nltk.word_tokenize(text) # tokenize text
    tokens = [w for w in tokens if not w in fillers] # remove fillers
    text_cleaned = ' '.join(tokens)
    return text_cleaned

def remove_comma_space(text):
    text = text.replace(', ', ' ')
    return text

def remove_hanging_punct(text):
    text = re.sub('\?+', '?', text)
    text = text.replace('. .', '.').replace('. ,', '.').replace(': .', ':').replace(' .', '.')
    if text.startswith('.'):
        text = text.strip('. ')
    text = re.sub('\s+([\.\,\:\?\(\)!\[\]])', r"\g<1>", text)
    text = re.sub(':\s+, ', ': ', text)
    return text

def remove_unwanted_punct(text):
    text = re.sub("""[\(\[]\.+[\)\]]|\[ ?unv\.?]|['`«»е]|\(? ?…\)?""", '', text)
    text = text.replace('„', '"').replace('“', '"')
    return text

def remove_dots(text):
    text = text.replace('...', '.').replace('..', '.')
    return text

def remove_extra_spaces(text):
    text = re.sub('[ \t]+', ' ', text)
    text = re.sub('\n+', '\n', text)
    return text

def fix_questions(text):
    q_errors = {
        'T:\n': '',
        'P:\n': '',
        'T: Hat sich das im Nachhinein gelöst? \nP:' : '',
        'T: Klappt das im Traum dass sie sich dann auch manchmal ein bisschen zur Wehr setzen? \nP:': '', 
        'T: Was ist dann da? \nP:': '',
        'T: Sind das ganz bunte Bilder? \nP:': '',
        

        'T: Fällt Ihnen eine Situation ein in der Sie Freude erlebt haben?': 'T: Schildern Sie eine Situation in der Sie Freude empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Wut erlebt haben?': 'T: Schildern Sie eine Situation in der Sie Wut empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Angst erlebt haben?': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Traurigkeit erlebt haben?': 'T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Freude empfunden haben?': 'T: Schildern Sie eine Situation in der Sie Freude empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Wut empfunden haben?': 'T: Schildern Sie eine Situation in der Sie Wut empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Angst empfunden haben?': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.', 
        'T: Fällt Ihnen eine Situation ein in der Sie Traurigkeit empfunden haben?': 'T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben.', 

        'T: Warum haben Sie in dieser Situation Freude erlebt?': 'T: Warum haben Sie in dieser Situation Freude empfunden?', 
        'T: Warum haben Sie in dieser Situation Angst erlebt?': 'T: Warum haben Sie in dieser Situation Angst empfunden?', 
        'T: Warum haben Sie in dieser Situation Traurigkeit erlebt?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum haben Sie in dieser Situation Wut erlebt?': 'T: Warum haben Sie in dieser Situation Wut empfunden?', 

        'T: Was gehört denn zu erleben von Angst dazu?': 'T: Warum haben Sie in dieser Situation Wut empfunden?', 
        'T: Was bedeutet denn sauer sein?': 'T: Was bedeutet Wut für Sie?', 


        'T: Beschreiben Sie eine Situation in der Sie Wut empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Wut empfunden haben.', 
        'T: Können Sie uns eine Situation schildern in der Sie Wut empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Wut empfunden haben.', 
        'T: Beschreiben Sie eine Situation in der Sie Angst empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.',  
        'T: Können Sie auch eine Situation beschreiben in der Sie Angst empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.', 
        'T: Können Sie mir eine Situation schildern in der Sie Angst empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.',  
        'T: Schildern Sie eine Situation in der Angst empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Angst empfunden haben.', 
        'T: Beschreiben Sie eine Situation in der Sie Traurigkeit empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben.', 
        'T: Und könnten Sie eine Situation schildern in der Sie Traurigkeit empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben.', 
        'T: Schildern Sie bitte eine Situation in der Sie Traurigkeit empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben.', 
        'T: Beschreiben Sie eine Situation in der Sie Freude empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Freude empfunden haben.', 
        'T: Schildern Sie eine Situation ein in der Sie Freude empfunden haben.': 'T: Schildern Sie eine Situation in der Sie Freude empfunden haben.', 

        'T: Warum haben Sie in diesen Situationen Traurigkeit empfunden?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum haben Sie in dieser Situation mit der Oma Traurigkeit erlebt?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?',  
        'T: Da waren Sie traurig? Warum haben Sie in dieser Situation Traurigkeit erlebt?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum haben Sie in dieser Situation Trauer empfunden?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum waren Sie in der Situation traurig?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum waren Sie wütend in der Situation?': 'T: Warum haben Sie in dieser Situation Wut empfunden?', 
        'T: Warum waren Sie in der Situation wütend?': 'T: Warum haben Sie in dieser Situation Wut empfunden?', 
        'T: Haben Sie beantwortet. Warum haben Sie in dieser Situation Wut erlebt?': 'T: Warum haben Sie in dieser Situation Wut empfunden?',   
        'T: Warum haben Sie in er Situation Angst empfunden?': 'T: Warum haben Sie in dieser Situation Angst empfunden?', 
        'T: Warum haben Sie denn in der Situation Angst empfunden?': 'T: Warum haben Sie in dieser Situation Angst empfunden?', 
        'T: Warum haben Sie in dieser Situation Angst?': 'T: Warum haben Sie in dieser Situation Angst empfunden?', 
        'T: Warum haben Sie sich in der Situation so gefreut?': 'T: Warum haben Sie in dieser Situation Freude empfunden?', 
        'T: Warum haben Sie sich in der Situation gefreut?': 'T: Warum haben Sie in dieser Situation Freude empfunden?', 
        'T: Warum empfinden Sie in dieser Situation dann Freude?': 'T: Warum haben Sie in dieser Situation Freude empfunden?', 
        'T: Warum haben Sie in dieser Situation Wut empfunden? Weil Sie nicht durchgehalten haben sagten Sie?': 'T: Warum haben Sie in dieser Situation Wut empfunden?',
        'T: Warum haben Sie in der Sitaution Traurigkeit empfunden?': 'T: Warum haben Sie in dieser Situation Traurigkeit empfunden?', 
        'T: Warum haben Sie in dieser Situation Wut empfunden ': 'T: Warum haben Sie in dieser Situation Wut empfunden?',
    
        'T: Was bedeutet denn Freude für Sie?': 'T: Was bedeutet Freude für Sie?', 
        'T: Und was bedeutet Wut für Sie?': 'T: Was bedeutet Wut für Sie?', 
        'T: Was bedeutet für Sie Angst?': 'T: Was bedeutet Angst für Sie?', 
        'T: Was bedeutet Trauigkeit für Sie?': 'T: Was bedeutet Traurigkeit für Sie?', 
        'T: Wie merken Sie dann die Wut?': 'T: Was bedeutet Wut für Sie?', 
        'T: Was bedeutet das denn? Traurig zu sein?': 'T: Was bedeutet Traurigkeit für Sie?', 
        'T: Was bedeutet Traurigkeit für Sie? Können Sie das ein bisschen beschreiben?': 'T: Was bedeutet Traurigkeit für Sie?', 
    }
    patterns = {
        'T: Und könnten Sie eine Situation schildern,': 'T: Schildern Sie eine Situation', 
        'T: Schildern Sie bitte eine Situation,': 'T: Schildern Sie eine Situation', 
        'T: Können Sie auch eine Situation beschreiben,': 'T: Schildern Sie eine Situation', 
        'T: Können Sie eine Situation schildern': 'T: Schildern Sie eine Situation', 
        'T: Und können Sie eine Situation schildern': 'T: Schildern Sie eine Situation', 
        'T: Beschreiben Sie eine Situation,': 'T: Schildern Sie eine Situation', 

        'T: Warum haben Sie in der Situation': 'T: Warum haben Sie in dieser Situation', 

        'in der Sie Freude empfunden haben?': 'in der Sie Freude empfunden haben.', 
        'in der Sie Wut empfunden haben?': 'in der Sie Wut empfunden haben.', 
        'in der Sie Angst empfunden haben?': 'in der Sie Angst empfunden haben.', 
        'in der Sie Traurigkeit empfunden haben?': 'in der Sie Traurigkeit empfunden haben.'
    }
    text = multiple_replace(text, q_errors)
    text = multiple_replace(text, patterns)
    return text


def replace_questions(text):
    text = text.replace('T: Was bedeutet Freude für Sie? P:', '\n\nhappy1:')
    text = text.replace('T: Was bedeutet Wut für Sie? P:', '\n\nanger1:')
    text = text.replace('T: Was bedeutet Angst für Sie? P:', '\n\nfear1:')
    text = text.replace('T: Was bedeutet Traurigkeit für Sie? P:', '\n\nsad1:')
    text = text.replace('T: Schildern Sie eine Situation in der Sie Freude empfunden haben. P:', '\n\nhappy2:')
    text = text.replace('T: Schildern Sie eine Situation in der Sie Wut empfunden haben. P:', '\n\nanger2:')
    text = text.replace('T: Schildern Sie eine Situation in der Sie Angst empfunden haben. P:', '\n\nfear2:')
    text = text.replace('T: Schildern Sie eine Situation in der Sie Traurigkeit empfunden haben. P:', '\n\nsad2:')
    text = text.replace('T: Warum haben Sie in dieser Situation Freude empfunden? P:', '\n\nhappy3:')
    text = text.replace('T: Warum haben Sie in dieser Situation Wut empfunden? P:', '\n\nanger3:')
    text = text.replace('T: Warum haben Sie in dieser Situation Angst empfunden? P:', '\n\nfear3:')
    text = text.replace('T: Warum haben Sie in dieser Situation Traurigkeit empfunden? P:', '\n\nsad3:')
    return text


def fix_turn_beginnings(text):
    text = text.replace('B:', 'P:')
    return text

# Sometimes the interviewer says other things as well 
# (e.g. T: Was gehört denn zu erleben von Angst dazu?)
def remove_extra_qestions(text):
    text = re.sub('T: .+? P:', '', text)
    return text

#Function for replacing number digits to number words in German

def number_to_german_word(number):
    german_mapping = {
    '1': 'eins', '2': 'zwei', '3': 'drei', '4': 'vier', '5': 'fünf', '6': 'sechs', '7': 'sieben', '8': 'acht', '9': 'neun','10': 'zehn',
    '11': 'elf', '12': 'zwölf', '13': 'dreizehn', '14': 'vierzehn', '15': 'fünfzehn', '16': 'sechzehn', '17': 'siebzehn', '18': 'achtzehn', '19': 'neunzehn', '20': 'zwanzig',
    '21': 'einundzwanzig', '22': 'zweiundzwanzig', '23': 'dreiundzwanzig', '24': 'vierundzwanzig', '25': 'fünfundzwanzig', '26': 'sechsundzwanzig', '27': 'siebenundzwanzig', '28': 'achtundzwanzig','29': 'neunundzwanzig', '30': 'dreißig', 
    '40': 'vierzig', '50': 'fünfzig', '60': 'sechzig', '70': 'siebzig', '80': 'achtzig', '90': 'neunzig', '100': 'hundert',
    '1000': 'tausend', '2000': 'zweitausend', '3000': 'dreitausend', '4000': 'viertausend', '5000': 'fünftausend', 
    '6000': 'sechstausend', '7000': 'siebentausend', '8000': 'achttausend', '9000': 'neuntausend', '10000': 'zehntausend',
    '1930': 'neunzehnhundertdreißig', '1931': 'neunzehnhunderteinunddreißig', '1932': 'neunzehnhundertzweiunddreißig', '1933': 'neunzehnhundertdreiunddreißig', '1934': 'neunzehnhundertvierunddreißig', '1935': 'neunzehnhundertfünfunddreißig', '1936': 'neunzehnhundertsechsunddreißig','1937': 'neunzehnhundertsiebenunddreißig', '1938': 'neunzehnhundertachtunddreißig', '1939': 'neunzehnhundertneununddreißig', 
    '1940': 'neunzehnhundertvierzig', '1941': 'neunzehnhunderteinundvierzig', '1942': 'neunzehnhundertzweiundvierzig', '1943': 'neunzehnhundertdreiundvierzig', '1944': 'neunzehnhundertvierundvierzig',  '1945': 'neunzehnhundertfünfundvierzig', '1946': 'neunzehnhundertsechsundvierzig', '1947': 'neunzehnhundertsiebenundvierzig', '1948': 'neunzehnhundertachtundvierzig', '1949': 'neunzehnhundertneunundvierzig', 
    '1950': 'neunzehnhundertfünfzig', '1951': 'neunzehnhunderteinundfünfzig', '1952': 'neunzehnhundertzweiundfünfzig', '1953': 'neunzehnhundertdreiundfünfzig', '1954': 'neunzehnhundertvierundfünfzig', '1955': 'neunzehnhundertfünfundfünfzig', '1956': 'neunzehnhundertsechsundfünfzig', '1957': 'neunzehnhundertsiebenundfünfzig', '1958': 'neunzehnhundertachtundfünfzig', '1959': 'neunzehnhundertneunundfünfzig',
    '1960': 'neunzehnhundertsechszig', '1961': 'neunzehnhunderteinundsechszig', '1962': 'neunzehnhundertzweiundsechszig', '1963': 'neunzehnhundertdreiundsechszig', '1964': 'neunzehnhundertvierundsechszig', '1965': 'neunzehnhundertfünfundsechszig', '1966': 'neunzehnhundertsechsundsechszig', '1967': 'neunzehnhundertsiebenundsechszig', '1968': 'neunzehnhundertachtundsechszig', '1969': 'neunzehnhundertneunundsechszig', 
    '1970': 'neunzehnhundertsiebzig', '1971': 'neunzehnhunderteinundsiebzig', '1972': 'neunzehnhundertzweiundsiebzig', '1973': 'neunzehnhundertdreiundsiebzig', '1974': 'neunzehnhundertvierundsiebzig',
    '1975': 'neunzehnhundertfünfundsiebzig', '1976': 'neunzehnhundertsechsundsiebzig', '1977': 'neunzehnhundertsiebenundsiebzig', '1978': 'neunzehnhundertachtundsiebzig', '1979': 'neunzehnhundertneunundsiebzig', 
    '1980': 'neunzehnhundertachtzig', '1981': 'neunzehnhunderteinundachtzig', '1982': 'neunzehnhundertzweiundachtzig', '1983': 'neunzehnhundertdreiundachtzig', '1984': 'neunzehnhundertvierundachtzig', '1985': 'neunzehnhundertfünfundachtzig', '1986': 'neunzehnhundertsechsundachtzig', '1987': 'neunzehnhundertsiebenundachtzig', '1988': 'neunzehnhundertachtundachtzig', '1989': 'neunzehnhundertneunundachtzig', 
    '1990': 'neunzehnhundertneunziger', '1991': 'neunzehnhunderteinundneunzig', '1992': 'neunzehnhundertzweiundneunzig', '1993': 'neunzehnhundertdreiundneunzig', '1994': 'neunzehnhundertvierundneunzig', '1995': 'neunzehnhundertfünfundneunzig', '1996': 'neunzehnhundertsechsundneunzig', '1997': 'neunzehnhundertsiebenundneunzig', '1998': 'neunzehnhundertachtundneunzig', '1999': 'neunzehnhundertneunundneunzig',
    '2000': 'zweitausend', '2001': 'zweitausendeins', '2002': 'zweitausendzwei', '2003': 'zweitausenddrei', '2004': 'zweitausendvier', '2005': 'zweitausendfünf', '2006': 'zweitausendsechs', '2007': 'zweitausendsieben', '2008': 'zweitausendacht', '2009': 'zweitausendneun',
    '2010': 'zweitausendzehn', '2011': 'zweitausendeinszehn', '2012': 'zweitausendzwölf', '2013': 'zweitausenddreizehn', '2014': 'zweitausendvierzehn', '2015': 'zweitausendfünfzehn', '2016': 'zweitausendsechzehn', '2017': 'zweitausendsiebzehn', '2018': 'zweitausendachtzehn', '2019': 'zweitausendneunzehn',
    '2020': 'zweitausendzwanzig', '2021': 'zweitausendeinundzwanzig', '2022': 'zweitausendzweiundzwanzig', '2023': 'zweitausenddreiundzwanzig', '2024': 'zweitausendvierundzwanzig', '2025': 'zweitausendfünfundzwanzig', '2026': 'zweitausendsechsundzwanzig', '2027': 'zweitausendsiebenundzwanzig', '2028': 'zweitausendachtundzwanzig', '2029': 'zweitausendneunundzwanzig',
    '2030': 'zweitausenddreißig',
    # Add more if needed
}

    return german_mapping.get(str(number), str(number))

# Example usage:
#number = '10000'
#word = number_to_german_word(number)
#print(word)


def replace_digits_with_german_words(text):
    # Regular expression to find all digits in the text
    digit_pattern = re.compile(r'\b\d+\b')

    # Find all matches in the text
    matches = digit_pattern.findall(text)

    # Replace each match with its corresponding German word
    for match in matches:
        german_word = number_to_german_word(int(match))
        text = text.replace(match, german_word)

    return text


#Final function for preprocessing text, consider not running clean_stopwords or clean_fillers depending on analysis
def preprocess(text, char_map, fillers=[], stopwords=[]):
    text = replace_chars(text, char_map)
    text = remove_timestamps(text)
    text = fix_turn_beginnings(text)
    text = remove_hanging_punct(text)
    text = remove_comma_space(text)
    text = remove_extra_spaces(text)
    text = fix_questions(fix_questions(text))
#     text = remove_extra_qestions(text)
    text = clean_stopwords(text, stopwords=stopwords)
    text = clean_fillers(text, fillers=fillers)

    text = remove_hanging_punct(text)
    text = remove_unwanted_punct(text)
#     text = replace_digits_with_german_words(text)
    text = remove_dots(text)
    text = remove_extra_spaces(text)
    ok, non_unicode = test_unicode(text)
    if '\x00' in text:
        return ''
    assert ok, non_unicode
    text = text.strip().strip('\n').strip('1')
    text = replace_questions(text)
    return text

<h3>Testing the preprocessing</h3>

In [289]:
#open transcript with the correct encoding

# File path
file_path = IN_DIR + r'13_6M_manual.txt'

# Detect the encoding
detected_encoding = detect_encoding(file_path)

# Open the file with the detected encoding
with open(file_path, encoding=detected_encoding) as f:
    raw = f.read()

In [290]:
print(raw)


T: Was bedeutet Traurigkeit für Sie?  #00:00:05-2#

P: Traurigkeit bedeutet für mich einen Verlust. Einen Verlust oder Reue. #00:00:20-2#

T: Können Sie eine Situation schildern, in der Sie Traurigkeit empfunden haben?   #00:00:25-4#

P: Ja, ich habe meine Kinder damals verloren. Und habe es bereut, warum ich sie verloren habe. Wieso das passiert ist. Traurigkeit ist auch, wenn man sich bewusst wird, dass Dinge mit einem gemacht wurden, die man. Also nein, das kann ich nicht so richtig nein. Aber wenn einem bewusst wird, dass etwas nicht richtig ist. Also wenn man es nicht versteht. Das kann ich jetzt nicht richtig beschreiben. Aber Traurigkeit ist traurig. #00:01:05-2#

T: Warum haben Sie in dieser Situation Traurigkeit empfunden?  #00:01:19-2#

P: Ja. Traurigkeit ist auch ein Gefühl könnte man sagen. Und ja weil ich starke Emotionen halt entwickelt habe für meine Kinder und deswegen.  #00:01:36-1#

T: Was bedeutet Angst für Sie?  #00:01:43-9#

P: Ja Angst. Angst, es bedeutet, dass m

In [292]:
cleaned = preprocess(raw, char_map=encoding_errors_dict)
print(cleaned)



sad1: Traurigkeit bedeutet für mich einen Verlust. Einen Verlust oder Reue. 

sad2: Ja ich habe meine Kinder damals verloren. Und habe es bereut warum ich sie verloren habe. Wieso das passiert ist. Traurigkeit ist auch wenn man sich bewusst wird dass Dinge mit einem gemacht wurden die man. Also nein das kann ich nicht so richtig nein. Aber wenn einem bewusst wird dass etwas nicht richtig ist. Also wenn man es nicht versteht. Das kann ich jetzt nicht richtig beschreiben. Aber Traurigkeit ist traurig. 

sad3: Ja. Traurigkeit ist auch ein Gefühl könnte man sagen. Und ja weil ich starke Emotionen halt entwickelt habe für meine Kinder und deswegen. 

fear1: Ja Angst. Angst es bedeutet dass man in die Enge geführt wird beziehungsweise bedroht wird. Vielleicht sogar erpresst oder irgendwas. Angst ist ein Gefühl. Also man zieht sich körperlich total man kehrt in sich ein. Man zittert vielleicht sogar auch. 

fear2: Ja also wenn mir jemand auflauert in meiner Wohnung jemand einbricht. Wenn ic

<h3>Preprocessing of all transcripts</h3>

In [294]:
# Function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        detector = chardet.universaldetector.UniversalDetector()
        for line in f:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
        return detector.result['encoding']

# Function to read a file with multiple encodings
def read_file_with_encodings(file_path, encodings):
    for encoding in encodings:
        try:
            with open(file_path, encoding=encoding) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError("Failed to decode the file with any of the specified encodings")

In [295]:
ids = []
transcripts = []

for (root, dirs, files) in os.walk(IN_DIR, topdown=True):
    for file in files:
        if file.endswith('.txt'):
            try:
                detected_encoding = detect_encoding(os.path.join(root, file))
                current_id = file.split('.')[0]  # Use [0] to get the part before the dot
                ids.append(current_id)
                raw = read_file_with_encodings(os.path.join(root, file), [detected_encoding, 'utf-8', 'latin-1'])
                cleaned = preprocess(raw, char_map=encoding_errors_dict)
                transcripts.append(cleaned)
            except (AssertionError, UnicodeDecodeError) as e:
                print(f"Error processing file: {file}")
                print(e)

# Save cleaned transcripts as new text files with id_c.txt format
for i, transcript in enumerate(transcripts):
    output_file_path = os.path.join(OUT_DIR_TRANSCRIPTS, f'{ids[i]}_c.txt')
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(transcript)

# NLP

In [296]:
ids = []
transcripts = []

for (root, dirs, files) in os.walk(OUT_DIR_TRANSCRIPTS, topdown=True):
    for file in files:
        try:
            detected_encoding = detect_encoding(os.path.join(root, file))
            current_id = file.split('.')[0]  # Use [0] to get the part before the dot
            ids.append(current_id)
            preprocessed = read_file_with_encodings(os.path.join(root, file), [detected_encoding, 'utf-8', 'latin-1'])
            transcripts.append(preprocessed)
        except (AssertionError, UnicodeDecodeError) as e:
            print(f"Error processing file: {file}")
            print(e)

df = pd.DataFrame(index=ids)
df['preprocessed_transcript'] = transcripts
df.head()

,preprocessed_transcript
45_6M_manual_c,\n\nsad1: Ja was bedeutet Traurigkeit. Also ge...
61_6M_manual_c,\n\nsad1: Ähm. Was bedeutet Traurigkeit für mi...
26_6M_manual_c,\n\nsad1: Äh dass jemand von mir Abschied geno...
53_6M_manual_c,\n\nsad1: Ja Traurigkeit bedeutet wenn man wen...
29_6M_manual_c,\n\nsad1: Also wenn man viel über was nachdenk...


In [298]:
df['preprocessed_transcript'][df['preprocessed_transcript'].apply(lambda text: 'P:' in text)]

Series([], Name: preprocessed_transcript, dtype: object)

### Split into questions

In [299]:
def split_into_questions(interview):
    # Define emotion-related questions and their corresponding identifiers
    emotion_questions = {
        'anger': ['anger1:', 'anger2:', 'anger3:'],
        'fear': ['fear1:', 'fear2:', 'fear3:'],
        'sadness': ['sad1:', 'sad2:', 'sad3:'],
        'happiness': ['happy1:', 'happy2:', 'happy3:']
    }
    
    # Map indexes onto emotions, not the other way around
    qids_emotions = {} 

    # Find the positions (indices) of emotion-related questions in the interview
    # The same question may be asked several times
    for emotion, questions in emotion_questions.items():
        for q in questions:
            if q in interview:
                ids = [m.start() for m in re.finditer(q, interview)]
                for i in ids:
                    # If several times the same question is asked, 
                    # they are assigned to the same emotion 
                    qids_emotions[i] = emotion

    # Initialize containers for emotion-related text segments
    emotion_texts = {emotion: '' for emotion in emotion_questions}
    
    sorted_qids = sorted(qids_emotions.keys())
    
    # Go through all qestion indicies
    for i, qid in enumerate(sorted_qids):
        # Look up which emotion this idex is
        emotion = qids_emotions[qid]
        if i + 1 < len(sorted_qids):
            next_qid = sorted_qids[i + 1]
            emotion_texts[emotion] += interview[qid:next_qid]
        else:
            emotion_texts[emotion] += interview[qid:]

    return emotion_texts

In [300]:
split_into_questions(cleaned)

{'anger': 'anger1: Wut ist eine Kraft. Eine spezifische. Und auch eine Kraft Menschen zu schützen und sie gleichzeitig zu besiegen. Da sind glaube ich zwei Facetten der Wut. Ich habe halt wie gesagt immer wieder so Kämpferphantasien die ich aber tatsächlich nicht auslebe. Und ich glaube dass die Wut sehr zentral für mich ist. Dass ich sehr viel Wut aufgestaut habe. Und dass sie auch für meine Zukunft so eine Art Taktgeber sein wird. Dass ich darüber eben eine Kraft aufbaue und die Wut halt auch rauslasse. \n\nanger2: Ne ähm es ist immer. Diese Phantasien die sind auch relativ unabhängig oft von dem was meine Umfeld gerade so gibt. Na gut doch. Meine Mutter da bin ich öfters sehr wütend. Da aber fällt es mir sehr schwer die Wut rauszulassen. Weil ich halt auch über meine Mutter da so geprägt bin. Die hatte selber immer so Ausraster. Und dass ich da so mich in mein Schneckenhaus zurückziehe. Und das ist eben auch so eine Sache der Wut. Dass ich die eben nutze um aus dieser Einschüchterun

In [301]:
questions_splits = []
for i, transcript in zip(ids, transcripts):
    questions_splits.append(split_into_questions(transcript))

df = pd.concat([df, pd.DataFrame(questions_splits, index=ids)], axis=1)
df.head()

,preprocessed_transcript,anger,fear,sadness,happiness
45_6M_manual_c,\n\nsad1: Ja was bedeutet Traurigkeit. Also ge...,anger1: Ja hm. Okay. Wut beruht auf Unsicherhe...,fear1: Naja. Also es ist halt so wenn nach dem...,sad1: Ja was bedeutet Traurigkeit. Also genere...,happy1: Naja. Mh. Ja das ist halt das Anstrebe...
61_6M_manual_c,\n\nsad1: Ähm. Was bedeutet Traurigkeit für mi...,anger1: Ja. Was bedeutet Wut für mich? Wut? Ja...,fear1: Ja das mir irgendwas passiert. Ja. \n\n...,sad1: Ähm. Was bedeutet Traurigkeit für mich? ...,happy1: Ja ähm wenn ich ähm. Ja weiß nicht. Wa...
26_6M_manual_c,\n\nsad1: Äh dass jemand von mir Abschied geno...,anger1: Ist ein unangenehmes Gefühl Wut. Also ...,fear1: Ein Zustand den ich nicht gern habe. Ei...,sad1: Äh dass jemand von mir Abschied genommen...,happy1: Das sind Fragen. Ähm Freude. Frei sein...
53_6M_manual_c,\n\nsad1: Ja Traurigkeit bedeutet wenn man wen...,anger1: Ähm Wut ja Wut wenn sich alles so aufs...,fear1: Ja Angst bedeutet Gefahr äh große Gefah...,sad1: Ja Traurigkeit bedeutet wenn man wenn ma...,happy1: Freude bedeutet für mich so eine Art L...
29_6M_manual_c,\n\nsad1: Also wenn man viel über was nachdenk...,anger1: Wenn ich auf was sauer bin. Stinkig. K...,fear1: Wenn ich mich zurückhalte vor irgendwas...,sad1: Also wenn man viel über was nachdenkt un...,happy1: Wenn mein Partner nach Hause kommt. Ja...


In [302]:
# Convert the column to string type
df['anger'] = df['anger'].astype(str)
df['fear'] = df['fear'].astype(str)
df['sadness'] = df['sadness'].astype(str)
df['happiness'] = df['happiness'].astype(str)

In [303]:
# Replace prompts in emotion columns
df['anger'] = df['anger'].replace({'\n\n': '', 'anger1:': '', 'anger2:': '', 'anger3:': ''}, regex=True)
df['sadness'] = df['sadness'].replace({'\n\n': '', 'sad1:': '', 'sad2:': '', 'sad3:': ''}, regex=True)
df['happiness'] = df['happiness'].replace({'\n\n': '', 'happy1:': '', 'happy2:': '', 'happy3:': ''}, regex=True)
df['fear'] = df['fear'].replace({'\n\n': '', 'fear1:': '', 'fear2:': '', 'fear3:': ''}, regex=True)
df['preprocessed_transcript'] = df['preprocessed_transcript'].replace({'\n\n': '', 'happy1:': '', 'happy2:': '', 'happy3:': '', 'sad1:': '', 'sad2:': '', 'sad3:': '', 'fear1:': '', 'fear2:': '', 'fear3:': '', 'anger1:': '', 'anger2:': '', 'anger3:': ''}, regex=True)

In [304]:
# Apply tokenization to the whole transcript
for colname in ['anger', 'fear', 'sadness', 'happiness', 'preprocessed_transcript']:
    df[colname+'_token_sents'] = df[colname].apply(lambda x: sent_tokenize(x))

In [305]:
# Calculate number of tokenized sentences per emotion column
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    df[colname+'_token_sents_len'] = df[colname+'_token_sents'].apply(len)

# Calculate number of tokenized sentences for whole transcript
df['total_token_sents_len'] = sum([df[colname+'_token_sents_len'] for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']])

# Save the results to CSV
df[['anger_token_sents_len',
 'fear_token_sents_len',
 'sadness_token_sents_len',
 'happiness_token_sents_len',
 'total_token_sents_len']].to_csv(OUT_DIR + 'count_token_sents_0.csv')

In [312]:
df[['anger','fear', 'sadness', 'happiness', 'preprocessed_transcript']].to_csv(OUT_DIR + r'split_questions_cp_0.tsv', sep='\t')

In [313]:
csv = df[['anger', 'fear', 'sadness', 'happiness', 
          'preprocessed_transcript']].applymap(lambda text: 
                                               preprocess(text, fillers=fillers, char_map={})).to_csv(sep='\t')

with open(OUT_DIR + r'split_questions_cp_wo_fillers.tsv', 'w') as f:
    f.write(csv)

In [310]:
df.to_pickle(OUT_DIR + 'df.pkl')

In [311]:
df = pd.read_pickle(OUT_DIR + 'df.pkl')

# Spacy w2v Vectors

In [ ]:
# Load the German SpaCy model
nlp = spacy.load("de_core_news_lg")

In [ ]:
oov = set()

def vectorize_sent(text, oov=oov, stopwords=fillers):
    doc = nlp(text)
#     oovs = set(token.text for token in doc if (token.text not in stopwords
#                                                and token.is_oov
#                                                and token.pos_ not in ['PUNCT', 'NUM']))
#     oov |= oovs
    return [token.vector for token in doc if ((token.text not in stopwords
                                               and not token.is_oov
                                               and token.pos_ not in ['PUNCT', 'NUM']))]

def vectorize_turn_history(turn_history, stopwords=fillers):
    turns_sentences_word_vectors = [[vectorize_sent(s, stopwords=stopwords) for s in t] for t in turn_history]
    sentences_word_vectors = flatten(turns_sentences_word_vectors)
    sentence_vectors = [np.mean(t, axis=0) for t in sentences_word_vectors if t]
    return sentence_vectors

def vectorize_sents(sents, stopwords=fillers):
    sentences_word_vectors = [vectorize_sent(s, stopwords=stopwords) for s in sents]
    sentence_vectors = [np.mean(t, axis=0) for t in sentences_word_vectors if t]
    return sentence_vectors

In [ ]:
for colname in ['anger','fear','sadness','happiness']:
    df[colname+'_sent_vectors'] = df[colname+'_token_sents'].map(vectorize_sents)

In [ ]:
df['preprocessed_transcript_sent_vectors'] = df['preprocessed_transcript_token_sents'].map(vectorize_sents)

In [ ]:
df.to_csv(OUT_DIR + 'vec_df.csv')

In [ ]:
ex = vectorize_sents(df['fear_token_sents'][0])

In [ ]:
len(df['fear_token_sents'][0])

### Get tokens, total and unique

In [ ]:
def get_text_tokens(text, stopwords=fillers):
    doc = nlp(text)
    return [token.lemma_.lower() for token in doc if ((token.text not in stopwords
                                             and token.pos_ not in ['PUNCT', 'NUM']))]

In [ ]:
for colname in ['anger','fear','sadness','happiness']:
    df[colname+'_tokens'] = df[colname].apply(lambda x: get_text_tokens(' '.join(x)))

In [ ]:
for colname in ['anger','fear','sadness','happiness']:
    df[colname+'_tokens_len'] = df[colname+'_tokens'].apply(len)
    df[colname+'_tokens_unique'] = df[colname+'_tokens'].apply(lambda x: len(set(x)))

In [ ]:
df['total_tokens_wo_fillers'] = sum([df[colname+'_tokens_len'] for colname in ['anger','fear','sadness','happiness']])
df['unique_tokens_wo_fillers'] = sum([df[colname+'_tokens_unique'] for colname in ['anger','fear','sadness','happiness']])

In [ ]:
df[['anger_tokens_len',
 'anger_tokens_unique',
 'fear_tokens_len',
 'fear_tokens_unique',
 'sadness_tokens_len',
 'sadness_tokens_unique',
 'happiness_tokens_len',
 'happiness_tokens_unique',
 'unique_tokens_wo_fillers',
 'total_tokens_wo_fillers']].to_csv(OUT_DIR + 'total_tokens_with_stopwords_cp_0.csv')

### oov

In [ ]:
# Load the German SpaCy model
# nlp = spacy.load("de_core_news_lg")

# Create a set to store OOV words
# Uncomment to reset OOV
# oov = set()

# Create a defaultdict to store OOV words along with their corresponding indices
oov2id = defaultdict(list)

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Process the transcript using SpaCy
    doc = nlp(row['preprocessed_transcript'])

    # Find OOV words and update the set and defaultdict
    for token in doc:
        if token.is_oov and token.text not in ['PUNCT', 'NUM']:
            oov.add(token.text)
            oov2id[index].append(token.text)

# Create a DataFrame from the defaultdict
ids, oovs_for_ids = zip(*((k, ', '.join(v)) for k, v in oov2id.items()))
oov_df = pd.DataFrame(index=ids)
oov_df['oov'] = oovs_for_ids

# Save the DataFrame to a CSV file
oov_df.to_csv(OUT_DIR + 'oov_table_spacy_cp_0.csv')

# Print the OOV words
#print("OOV words:", oov)


## Metrics

In [ ]:
# I redefine cosine similarity because some standard implementations have precision errors 
def cos_sim(v1, v2):
    return np.inner(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

### Local Coherence
(let i = cosine distance between an averaged vector of the neighbouring clauses)
a plot:
x axis: number of the current clause
y axis: i

- the differences between the averages for the groups
- the differences between the averages for the averages of the groups

In [ ]:
def get_local_coherence(clause_vectors):
    if len(clause_vectors) <= 1:
        return [np.nan]
    local_coherence_list = []
    for i in range(len(clause_vectors)-1):
        local_coherence_list.append(cos_sim(clause_vectors[i], clause_vectors[i+1]))
    return local_coherence_list

In [ ]:
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    df[f'{colname}_lcoh'] = df[f'{colname}_sent_vectors' ][~pd.isnull(df[f'{colname}_sent_vectors'])].apply(get_local_coherence)
    df[f'{colname}_mean_lcoh'] = df[f'{colname}_lcoh'][~pd.isnull(df[f'{colname}_lcoh'])].apply(np.mean)
    df[f'{colname}_std_lcoh'] = df[f'{colname}_lcoh'][~pd.isnull(df[f'{colname}_lcoh'])].apply(np.std)

### Global coherence

(let us take a weighted average of each clause, then an average of these centroids in one text, then a average of the averages of all the texts at hand)

- cosine distance between the average of all the texts and the average of the current text for each participant
- cosine distance between the average of all the control texts and the average of the current text for each participant

In [ ]:
def compare_text_to_a_standard_vector(clause_vectors, standard_vector):
    """
    cosine similarity of a given vector and a list of vectors

    :param clause_vectors: list of np.arrays (vectors) or a np.array of shape (sent_len, emb_size)
    :param standard_vector: np.array of int or float of shape (emb_size)
    """
    average_file_vector = np.average(clause_vectors, axis=0)
    return cos_sim(average_file_vector, standard_vector)

In [ ]:
def get_standard_vectors(data, column_name):
    """
    gets average vector across all vectors in a givem column, unlisting them

    :param data: pd.Dataframe
    :param column_name: str, column with list of lists or np.arrays of int ot float of shape (sent_len, emb_size)
    :return: np.array of int or float of shape (emb_size)
    """
    column = data[column_name][~pd.isnull(data[column_name])]
    return np.nanmean(np.vstack(flatten(column.values)), axis=0)

In [ ]:
def get_global_coherence(clause_vectors, standard_vector):
    """
    calculates global coherence

    :param clause_vectors: list of np.arrays (vectors) or a np.array of shape (sent_len, emb_size)
    :param standard_vector:  np.array of int or float of shape (emb_size)
    :return: list of float of len sent_len, global coherence of each clasue
    """
    if len(clause_vectors) <= 1:
        return np.nan
    return [cos_sim(vec, standard_vector) for vec in clause_vectors]

In [ ]:
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    standard = get_standard_vectors(df, f'{colname}_sent_vectors')
    df[f'{colname}_gcoh'] = df[f'{colname}_sent_vectors'][~pd.isnull(df[f'{colname}_sent_vectors'])].apply(lambda vecs: get_global_coherence(vecs, standard))
    df[f'{colname}_mean_gcoh'] = df[f'{colname}_gcoh'][~pd.isnull(df[f'{colname}_gcoh'])].apply(np.mean)
    df[f'{colname}_std_gcoh'] = df[f'{colname}_gcoh'][~pd.isnull(df[f'{colname}_gcoh'])].apply(np.std)

In [ ]:
df[['anger_mean_lcoh', 'anger_std_lcoh','fear_mean_lcoh', 'fear_std_lcoh',
       'sadness_mean_lcoh', 'sadness_std_lcoh',
       'happiness_mean_lcoh', 'happiness_std_lcoh', 
       'preprocessed_transcript_mean_lcoh', 'preprocessed_transcript_std_lcoh',
    'anger_mean_gcoh',
       'anger_std_gcoh', 'fear_mean_gcoh', 'fear_std_gcoh',
       'sadness_mean_gcoh', 'sadness_std_gcoh',
       'happiness_mean_gcoh', 'happiness_std_gcoh',
   'preprocessed_transcript_mean_gcoh', 'preprocessed_transcript_std_gcoh']].to_csv(OUT_DIR + 'w2v_results_cp_0.csv')

In [ ]:
df[['anger_mean_lcoh', 'fear_mean_lcoh',
    'sadness_mean_lcoh', 'happiness_mean_lcoh', 
    'anger_mean_gcoh', 'fear_mean_gcoh', 
    'sadness_mean_gcoh', 'happiness_mean_gcoh']]

# GloVe vectors

In [ ]:
with open('vectors.txt', encoding='utf-8') as f:
    lines = f.readlines()

In [ ]:
def process_line(line):
    key, str_value = line.strip().split(' ', 1)
    return key, np.array([float(x) for x in str_value.split()])

In [ ]:
glove = {}
for line in lines:
    key, value = process_line(line)
    if test_unicode(key):
        glove[key] = value

In [ ]:
glove_oov = set()
def get_glove_vector(word):
    if word.lower() in glove:
        return glove[word.lower()]
    else:
        glove_oov.add(word)
        return None

In [ ]:
get_glove_vector('angst')

In [ ]:
def vectorize_sent_glove(text, oov=glove_oov):
    doc = nlp(text)
    oovs = set(token.text for token in doc if token.lemma_.lower() not in glove)
    oov |= oovs
    return [get_glove_vector(token.lemma_.lower()) for token in doc if (token.pos_ not in ['PUNCT', 'NUM'] and get_glove_vector(token.lemma_.lower()) is not None)]

def vectorize_turn_history_glove(turn_history):
    turns_sentences_word_vectors = [[vectorize_sent_glove(s) for s in t] for t in turn_history]
    sentences_word_vectors = flatten(turns_sentences_word_vectors)
    sentence_vectors = [np.mean(t, axis=0) for t in sentences_word_vectors if t]
    return sentence_vectors

def vectorize_sents_glove(sents, stopwords=fillers):
    sentences_word_vectors = [vectorize_sent_glove(s, oov=glove_oov) for s in sents]
    sentence_vectors = [np.mean(t, axis=0) for t in sentences_word_vectors if t]
    return sentence_vectors

In [ ]:
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    df[colname+'_glove_sent_vectors'] = df[colname+'_token_sents'].apply(vectorize_sents_glove)

In [ ]:
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    df[f'{colname}_glove_lcoh'] = df[f'{colname}_glove_sent_vectors' ][~pd.isnull(df[f'{colname}_glove_sent_vectors' ])].apply(get_local_coherence)
    df[f'{colname}_glove_mean_lcoh'] = df[f'{colname}_glove_lcoh'][~pd.isnull(df[f'{colname}_glove_lcoh'])].apply(np.mean)
    df[f'{colname}_glove_std_lcoh'] = df[f'{colname}_glove_lcoh'][~pd.isnull(df[f'{colname}_glove_lcoh'])].apply(np.std)

In [ ]:
df['anger_glove_sent_vectors']

In [ ]:
for colname in ['anger','fear','sadness','happiness', 'preprocessed_transcript']:
    standard = get_standard_vectors(df, f'{colname}_glove_sent_vectors')
    df[f'{colname}_glove_gcoh'] = df[f'{colname}_glove_sent_vectors'][~pd.isnull(df[f'{colname}_glove_sent_vectors'])].apply(lambda vecs: get_global_coherence(vecs, standard))
    df[f'{colname}_glove_mean_gcoh'] = df[f'{colname}_glove_gcoh'][~pd.isnull(df[f'{colname}_glove_gcoh'])].apply(np.mean)
    df[f'{colname}_glove_std_gcoh'] = df[f'{colname}_glove_gcoh'][~pd.isnull(df[f'{colname}_glove_gcoh'])].apply(np.std)

In [ ]:
df[['anger_glove_mean_lcoh', 'anger_glove_std_lcoh','fear_glove_mean_lcoh', 'fear_glove_std_lcoh',
       'sadness_glove_mean_lcoh', 'sadness_glove_std_lcoh',
       'happiness_glove_mean_lcoh', 'happiness_glove_std_lcoh',
   'preprocessed_transcript_glove_mean_lcoh', 'preprocessed_transcript_glove_mean_lcoh',
    'anger_glove_mean_gcoh',
       'anger_glove_std_gcoh', 'fear_glove_mean_gcoh', 'fear_glove_std_gcoh',
       'sadness_glove_mean_gcoh', 'sadness_glove_std_gcoh',
       'happiness_glove_mean_gcoh', 'happiness_glove_std_gcoh',
   'preprocessed_transcript_glove_mean_gcoh', 'preprocessed_transcript_glove_mean_gcoh']].to_csv(OUT_DIR + 'glove_results.csv')

In [ ]:
df.to_pickle(OUT_DIR + 'net.pkl')

In [ ]:
df.describe()

In [ ]:
df[['anger_glove_mean_lcoh', 'fear_glove_mean_lcoh', 'sadness_glove_mean_lcoh', 'happiness_glove_mean_lcoh', 
    'anger_glove_mean_gcoh', 'fear_glove_mean_gcoh', 'sadness_glove_mean_gcoh', 'happiness_glove_mean_gcoh']]